# Stations Discovery + Match-Up

Given a SENTINEL-3 OLCI, i.e. [S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004](https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc/items/S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004) with specified attributes:

- `geometry`,
- [`properties.start_datetime`, `properties.end_datetime`]

search all active AERONET stations, where:

- `Site_Latitude(Degrees),Site_Longitude(Degrees)` in `geometry`

for each discovered station, filter all acquisitions where:

- `properties.start_datetime` - 1h <= `Date(dd:mm:yyyy),Time(hh:mm:ss)` <= `properties.end_datetime` + 1h

In [7]:
from httpx import (
    Client,
    Response
)
from pathlib import Path
from pystac import Item
from typing import Any

import json
import sys

out_dir: Path = Path('.')

with Client() as client:
    response: Response = client.get('https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc/items/S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004')
    item_str: str = response.text
    item_dict = json.loads(item_str)

json.dump(item_dict, sys.stdout, indent=2)

{
  "id": "S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004",
  "bbox": [
    -1.51876,
    39.545,
    18.484,
    52.4535
  ],
  "type": "Feature",
  "links": [
    {
      "rel": "collection",
      "type": "application/json",
      "href": "https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc"
    },
    {
      "rel": "parent",
      "type": "application/json",
      "href": "https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc"
    },
    {
      "rel": "root",
      "type": "application/json",
      "href": "https://stac.dataspace.copernicus.eu/v1/"
    },
    {
      "rel": "self",
      "type": "application/geo+json",
      "href": "https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc/items/S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004"
    },
    {
      "rel": "version-history",
      "href": "https://trace.

## Execute the `query_stations_from_parquet` operation

In [8]:

source_item: Item = Item.from_dict(item_dict)
cql2_filter = {
    "op": "and",
    "args": [
        {
            "op": "s_intersects",
            "args": [
                {"property": "geometry"},
                source_item.geometry,
            ],
        }
    ],
}

json.dump(cql2_filter, sys.stdout, indent=2)

{
  "op": "and",
  "args": [
    {
      "op": "s_intersects",
      "args": [
        {
          "property": "geometry"
        },
        {
          "type": "Polygon",
          "coordinates": [
            [
              [
                -1.5187599999999999,
                41.97070000000001
              ],
              [
                13.725899999999998,
                39.545
              ],
              [
                18.484,
                49.88549999999999
              ],
              [
                0.10018799999999999,
                52.45349999999999
              ],
              [
                -1.5187599999999999,
                41.97070000000001
              ]
            ]
          ]
        }
      ]
    }
  ]
}

In [9]:
from pygeofilter_aeronet import (
    DEFAULT_STATIONS_PARQUET_URL,
    query_stations_from_parquet
)
from typing import List

_, resulting_items = query_stations_from_parquet(
    cql2_filter=cql2_filter,
    file_path=DEFAULT_STATIONS_PARQUET_URL
)

## Prepare the Map screen

In [ ]:
print(f"Found {len(resulting_items)} STAC Item(s) representing AERONET Station(s)")

for item in resulting_items:
    print(item.properties)

Found 114 STAC Item(s) representing AERONET Station(s)


In [11]:
from folium import (
    GeoJson,
    LayerControl,
    Map
)
from folium.plugins import Fullscreen

map: Map = Map()
layer_control = LayerControl(position="topright", collapsed=True)
fullscreen = Fullscreen()

GeoJson(
    source_item,
    name=source_item.id,
    style_function=lambda f: {
        "fillColor": "yellow",
        "color": "red",
        "weight": 2,
        "fillOpacity": 0.3,
    }
).add_to(map)

for item in resulting_items:
    GeoJson(
        item,
        name=item.id,
        style_function=lambda f: {
            "fillColor": "yellow",
            "color": "red",
            "weight": 2,
            "fillOpacity": 0.3,
        }
).add_to(map)

layer_control.add_to(map)
fullscreen.add_to(map)
map.fit_bounds(map.get_bounds()) # type: ignore not to important for the demo
map

## `search` data for each discovered station

In [12]:
from datetime import (
    datetime,
    timedelta,
    timezone
)
from dateutil import parser
from geopandas import read_parquet
from geopandas.geodataframe import GeoDataFrame
from IPython.display import (
    display,
    Markdown
)
from pygeofilter_aeronet import aeronet_search

def _parse_date_from_properties(date_string_id: str) -> datetime:
    return parser.parse(source_item.properties[date_string_id])

start_datetime = _parse_date_from_properties('start_datetime') - timedelta(hours=1)
end_datetime = _parse_date_from_properties('end_datetime') + timedelta(hours=1)

def _serialize_date(input_date: datetime) -> str:
    dt_utc = input_date.astimezone(timezone.utc)
    return dt_utc.isoformat().replace("+00:00", "Z")

map: Map = Map()
layer_control = LayerControl(position="topright", collapsed=True)
fullscreen = Fullscreen()

GeoJson(
    source_item,
    name=source_item.id,
    style_function=lambda f: {
        "fillColor": "yellow",
        "color": "red",
        "weight": 2,
        "fillOpacity": 0.3,
    }
).add_to(map)

DEFAULT_TIMEOUT: int = 5

FILTERING_SITES: List[str] = [ 'Lille', 'Paris', 'Rome_Tor_Vergata' ]

for item in resulting_items:
    site_name: str = item.properties['aeronet:site_name']

    if site_name in FILTERING_SITES:
        cql2_filter = {
            "op": "and",
            "args": [
                {"op": "eq", "args": [{"property": "site"}, site_name]},
                {"op": "eq", "args": [{"property": "data_type"}, "AOD10"]},
                {"op": "eq", "args": [{"property": "format"}, "csv"]},
                {"op": "eq", "args": [{"property": "data_format"}, "daily-average"]},
                {
                    "op": "t_after",
                    "args": [
                        {"property": "time"},
                        {"timestamp": _serialize_date(start_datetime)},
                    ],
                },
                {
                    "op": "t_before",
                    "args": [
                        {"property": "time"},
                        {"timestamp": _serialize_date(end_datetime)},
                    ],
                }
            ],
        }

        try:
            resulting_item: Item = aeronet_search(
                cql2_filter=cql2_filter,
                output_dir=out_dir,
                timeout=DEFAULT_TIMEOUT
            )

            display(Markdown(f"### {site_name} results as Data Frame"))

            geoparquet_file: str = resulting_item.get_assets()['geoparquet'].href
            geoparquet_data: GeoDataFrame = read_parquet(geoparquet_file)

            display(geoparquet_data)

            geoparquet_data = geoparquet_data.dissolve(by='AERONET_Site')

            GeoJson(
                geoparquet_data.to_json(default=str),
                name="Stac Item footprints",
                style_function=lambda x: {"fillColor": "#00000000", "color": "#0000ff", "weight": 1},
                control=True,
            ).add_to(map)
        except Exception as e:
            print(f"An error occurred while retrieving data from site {site_name}: {e}")

layer_control.add_to(map)
fullscreen.add_to(map)
map.fit_bounds(map.get_bounds()) # type: ignore not to important for the demo
map

2025-11-18 17:42:30.556 | SUCCESS  | pygeofilter_aeronet:aeronet_search:248 - Query on https://aeronet.gsfc.nasa.gov successfully obtained data:
2025-11-18 17:42:30.561 | SUCCESS  | pygeofilter_aeronet:aeronet_search:261 - Data saved to to CSV file: /home/stripodi/Documents/pygeofilter/pygeofilter-aeronet/docs/use_cases/3c413a12-ec5e-40b3-8b01-84031088fed7.csv
2025-11-18 17:42:30.573 | SUCCESS  | pygeofilter_aeronet:aeronet_search:295 - Data saved to GeoParquet file: /home/stripodi/Documents/pygeofilter/pygeofilter-aeronet/docs/use_cases/3c413a12-ec5e-40b3-8b01-84031088fed7.parquet


### Lille results as Data Frame

,AERONET_Site,Date(dd:mm:yyyy),Time(hh:mm:ss),Day_of_Year,AOD_1640nm,AOD_1020nm,AOD_870nm,AOD_865nm,AOD_779nm,AOD_675nm,...,N[340-440_Angstrom_Exponent],N[440-675_Angstrom_Exponent[Polar]],Data_Quality_Level,AERONET_Instrument_Number,AERONET_Site_Name,Site_Latitude(Degrees),Site_Longitude(Degrees),Site_Elevation(m),geometry,datetime
0,Lille,30:10:2025,12:00:00,303,0.294629,0.309954,0.303359,-999.0,-999.0,0.296156,...,45,0,lev10,1345,Lille,50.611667,3.141667,60.0,POINT (3.14167 50.61167),2025-10-30 12:00:00
1,Lille,31:10:2025,12:00:00,304,0.546914,0.518839,0.499880,-999.0,-999.0,0.486531,...,5,0,lev10,1345,Lille,50.611667,3.141667,60.0,POINT (3.14167 50.61167),2025-10-31 12:00:00


2025-11-18 17:42:31.279 | SUCCESS  | pygeofilter_aeronet:aeronet_search:248 - Query on https://aeronet.gsfc.nasa.gov successfully obtained data:
2025-11-18 17:42:31.282 | SUCCESS  | pygeofilter_aeronet:aeronet_search:261 - Data saved to to CSV file: /home/stripodi/Documents/pygeofilter/pygeofilter-aeronet/docs/use_cases/a11b6ecb-67ef-4dd7-8f16-5bf84db188a4.csv
2025-11-18 17:42:31.293 | SUCCESS  | pygeofilter_aeronet:aeronet_search:295 - Data saved to GeoParquet file: /home/stripodi/Documents/pygeofilter/pygeofilter-aeronet/docs/use_cases/a11b6ecb-67ef-4dd7-8f16-5bf84db188a4.parquet


### Paris results as Data Frame

,AERONET_Site,Date(dd:mm:yyyy),Time(hh:mm:ss),Day_of_Year,AOD_1640nm,AOD_1020nm,AOD_870nm,AOD_865nm,AOD_779nm,AOD_675nm,...,N[340-440_Angstrom_Exponent],N[440-675_Angstrom_Exponent[Polar]],Data_Quality_Level,AERONET_Instrument_Number,AERONET_Site_Name,Site_Latitude(Degrees),Site_Longitude(Degrees),Site_Elevation(m),geometry,datetime
0,Paris,30:10:2025,12:00:00,303,0.153396,0.151924,0.148786,-999.0,-999.0,0.147329,...,30,0,lev10,718,Paris,48.846797,2.355508,50.0,POINT (2.35551 48.8468),2025-10-30 12:00:00
1,Paris,31:10:2025,12:00:00,304,0.133803,0.132780,0.130804,-999.0,-999.0,0.133111,...,7,0,lev10,718,Paris,48.846797,2.355508,50.0,POINT (2.35551 48.8468),2025-10-31 12:00:00


2025-11-18 17:42:32.308 | SUCCESS  | pygeofilter_aeronet:aeronet_search:248 - Query on https://aeronet.gsfc.nasa.gov successfully obtained data:
2025-11-18 17:42:32.311 | SUCCESS  | pygeofilter_aeronet:aeronet_search:261 - Data saved to to CSV file: /home/stripodi/Documents/pygeofilter/pygeofilter-aeronet/docs/use_cases/b83ae5d3-9e3f-4546-b648-aa76aa4f66ba.csv
2025-11-18 17:42:32.320 | SUCCESS  | pygeofilter_aeronet:aeronet_search:295 - Data saved to GeoParquet file: /home/stripodi/Documents/pygeofilter/pygeofilter-aeronet/docs/use_cases/b83ae5d3-9e3f-4546-b648-aa76aa4f66ba.parquet


### Rome_Tor_Vergata results as Data Frame

,AERONET_Site,Date(dd:mm:yyyy),Time(hh:mm:ss),Day_of_Year,AOD_1640nm,AOD_1020nm,AOD_870nm,AOD_865nm,AOD_779nm,AOD_675nm,...,N[340-440_Angstrom_Exponent],N[440-675_Angstrom_Exponent[Polar]],Data_Quality_Level,AERONET_Instrument_Number,AERONET_Site_Name,Site_Latitude(Degrees),Site_Longitude(Degrees),Site_Elevation(m),geometry,datetime
0,Rome_Tor_Vergata,30:10:2025,12:00:00,303,0.626251,0.602303,0.591329,-999.0,-999.0,0.586273,...,11,0,lev10,1321,Rome_Tor_Vergata,41.83955,12.647333,130.0,POINT (12.64733 41.83955),2025-10-30 12:00:00
1,Rome_Tor_Vergata,31:10:2025,12:00:00,304,0.062751,0.081665,0.088833,-999.0,-999.0,0.106361,...,36,0,lev10,1321,Rome_Tor_Vergata,41.83955,12.647333,130.0,POINT (12.64733 41.83955),2025-10-31 12:00:00
